Url I got the below info from https://www.yelp.com/developers/v3/manage_app?app_created=True

With just API Keys the process to authenticate is:

Get your API Key from the Manage App page.
Put the API Key in the request header as "Authorization: Bearer <YOUR API KEY>".

In [1]:
import requests
import json
import pandas as pd
import time

In [2]:
file = open("Yelp_API_Key.txt","r")
api_key = file.read()
file.close()

In [3]:
headers = {'Authorization': 'Bearer %s' % api_key}

def business_search(location, term):
    yelp_review_list = []
    endpoint_url = "https://api.yelp.com/v3/businesses/search"
    for offset in range(0, 1000, 50):
        params = {
        'limit': 50,
        'term': term.replace(' ', '+'),  #should be restaurants intially
        'location': location.replace(' ', '+'),
        'offset': offset
        }
        time.sleep(2)
        results = requests.get(endpoint_url, params = params, headers=headers)
        yelp_review_list.append(pd.json_normalize(results.json()['businesses']))
    business_df = pd.concat(yelp_review_list)
    return(business_df)

def buiness_reviews(yelp_id):
    review_df_list = []
    endpoint_url = f"https://api.yelp.com/v3/businesses/{yelp_id}/reviews"
    results = requests.get(endpoint_url, headers=headers)
    try:
        review_df = pd.json_normalize(results.json()['reviews'])
        return(review_df)
    except:
        print('No review for ID' + yelp_id)
    

In [4]:
# yelp_id = 'fqIYC7BIVIeZahiDpdZhZg'
# endpoint_url = f"https://api.yelp.com/v3/businesses/{yelp_id}/reviews"
# results = requests.get(endpoint_url, headers=headers)
# results.json()['reviews']

In [5]:
Missoula_rest = business_search('Missoula', 'restaurants')

In [6]:
Missoula_rest.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,fqIYC7BIVIeZahiDpdZhZg,the-trough-missoula,The Trough,https://s3-media3.fl.yelpcdn.com/bphoto/kUfipT...,False,https://www.yelp.com/biz/the-trough-missoula?a...,111,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.3,"[delivery, pickup]",...,46.850959,-114.080624,2106 Clements Rd,,,Missoula,59804,US,MT,"[2106 Clements Rd, Missoula, MT 59804]"
1,77TINct71T-TYV3m-RIY4Q,brasserie-porte-rouge-missoula,Brasserie Porte Rouge,https://s3-media3.fl.yelpcdn.com/bphoto/5ZtA_a...,False,https://www.yelp.com/biz/brasserie-porte-rouge...,39,"[{'alias': 'brasseries', 'title': 'Brasseries'...",4.5,[],...,46.869390,-113.992780,231 E Front St,,None,Missoula,59802,US,MT,"[231 E Front St, Missoula, MT 59802]"
2,2mChJfBZEfXAYNNza2B4lw,gild-missoula-2,Gild,https://s3-media1.fl.yelpcdn.com/bphoto/_V2TmS...,False,https://www.yelp.com/biz/gild-missoula-2?adjus...,88,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.4,[delivery],...,46.866588,-113.997832,515 S Higgins Ave,,None,Missoula,59801,US,MT,"[515 S Higgins Ave, Missoula, MT 59801]"
3,90kbM8J48c_3qyO2hzHKyw,plonk-missoula,Plonk,https://s3-media3.fl.yelpcdn.com/bphoto/LtXjHT...,False,https://www.yelp.com/biz/plonk-missoula?adjust...,330,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,[delivery],...,46.872736,-113.993209,322 N Higgins Ave,,,Missoula,59802,US,MT,"[322 N Higgins Ave, Missoula, MT 59802]"
4,5tlg-MSf7ZCiLvvAXNmUoQ,tamarack-brewing-company-missoula,Tamarack Brewing Company,https://s3-media2.fl.yelpcdn.com/bphoto/BItNeJ...,False,https://www.yelp.com/biz/tamarack-brewing-comp...,787,"[{'alias': 'sportsbars', 'title': 'Sports Bars...",4.1,"[delivery, pickup]",...,46.870967,-113.997316,231 W Front St,,,Missoula,59802,US,MT,"[231 W Front St, Missoula, MT 59802]"


In [7]:
Missoula_rest.drop(['alias', 'image_url', 'is_closed', 'display_phone', 'distance', 'categories', 'transactions', 'coordinates.latitude',
                    'coordinates.longitude', 'location.address3', 'location.country', 'location.state', 
                    'location.display_address'], axis=1, inplace=True)

In [8]:
Missoula_rest.rename(columns = {'location.zip_code':'Zip_Code', 
                              'location.city':'City',
                              'location.address2': 'Address_2',
                              'location.address1' : 'Address'}, inplace = True)

In [9]:
Missoula_rest = Missoula_rest.set_index('name')

In [10]:
Missoula_rest.head()

,id,url,review_count,rating,price,phone,Address,Address_2,City,Zip_Code
name,,,,,,,,,,
The Trough,fqIYC7BIVIeZahiDpdZhZg,https://www.yelp.com/biz/the-trough-missoula?a...,111,4.3,$$,+14067213322,2106 Clements Rd,,Missoula,59804
Brasserie Porte Rouge,77TINct71T-TYV3m-RIY4Q,https://www.yelp.com/biz/brasserie-porte-rouge...,39,4.5,NaN,+14065410231,231 E Front St,,Missoula,59802
Gild,2mChJfBZEfXAYNNza2B4lw,https://www.yelp.com/biz/gild-missoula-2?adjus...,88,4.4,NaN,+14069263258,515 S Higgins Ave,,Missoula,59801
Plonk,90kbM8J48c_3qyO2hzHKyw,https://www.yelp.com/biz/plonk-missoula?adjust...,330,4.3,$$,+14069261791,322 N Higgins Ave,,Missoula,59802
Tamarack Brewing Company,5tlg-MSf7ZCiLvvAXNmUoQ,https://www.yelp.com/biz/tamarack-brewing-comp...,787,4.1,$$,+14068303113,231 W Front St,,Missoula,59802


In [11]:
#cleaning dataset to ensure only missoula, mt locations
Missoula_rest = Missoula_rest[Missoula_rest['City'] == 'Missoula']
Missoula_rest = Missoula_rest[Missoula_rest['Address'] != '']

In [12]:
review_dictionary_yelp = {}
for key, value in Missoula_rest.iterrows():
    review_dictionary_yelp[key] = buiness_reviews(value['id'])

No review for ID6yLd2vaSaWqTqWIbLEoiVA
No review for ID_twe3il6IUIu-NhpwRxBIg
No review for IDKTUDSn78F8G9ySBbMStrTQ
No review for IDmk_8Co2W3J-q9-rc_j4bww
No review for IDZ1fvMq9z_oLiDPgubFs5oA
No review for IDIBo4QhrVuM984LESLwfZTg
No review for IDlJk073s8py--ghDlftT3lQ
No review for ID3A7hrrpIapTXAxdTjSEFSg
No review for IDR-c4TG0oWqFFY7032Dw03w
No review for IDi5tSQxZp3VBUU9sQA-3VHQ
No review for IDduMIMFFOzGA3ntVtbB-1dA
No review for IDhlyjPH_P9XPufKeI-y176g
No review for IDFCIlRyCSh0tD6lU0J8EKyA
No review for IDvAuNMQQ9Cy1-kb8QvIKLhg
No review for IDrklIAWtA181CDo-2Di8liA
No review for IDU0nWAh1CPgqug33R_lUJAQ
No review for IDSxH61nNgt692GKbM_di06A
No review for IDVoK4bCyugbKHFQS42dvPgw
No review for IDvw6FK3ay7DuvE17ZUdi3Hw
No review for IDGCSkR7BiF89YIgkkej7qAQ
No review for IDNPG7YHZAIWeO8qKKIVVBTg
No review for IDwAmqjvUaTy1bPd4yuSDi2g
No review for IDVEBa6m8jOHMWWSTaOsqsnw
No review for IDKfdjc2D8D8hcsfmnS_0kvA
No review for IDJO5fqqd61a1Sp1bgxafSUQ


In [14]:
for key in review_dictionary_yelp:
    try:
        review_dictionary_yelp[key].drop(['id', 'url', 'user.id', 'user.profile_url', 'user.image_url'],
                                         axis=1, inplace=True)
        review_dictionary_yelp[key].rename(columns = {'user.name':'Name'}, inplace = True)
    except:
        continue

In [15]:
review_dictionary_yelp['Boxcar Bistro']

,text,rating,time_created,Name
0,Good bar and service \n\n\nI love it you good ...,5,2024-02-07 10:05:33,Jean G.
1,This place stinks. Very dirty in the kitchen....,1,2024-02-06 19:45:05,Bryan G.
2,"Cute little Bistro, great decor with chandelie...",5,2022-02-02 20:32:00,Randy P.


In [ ]:
# for key in review_dictionary_yelp: 
    # review_dictionary_yelp[key].to_csv('Yelp_Reviews/' + str(key) + '_Yelp_reviews.csv')

In [ ]:
# Missoula_rest.to_csv('Yelp_Missoula_Restaurants.csv')